In [34]:
# 사용할 패키지 임포트

In [35]:
from PIL import Image
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [36]:
# png 파일 jpg파일로 변환하기

def convert_png_to_jpg(path) : 
    # jpg 파일을 저장하기위한 디렉토리의 생성
    if not os.path.exists(path + '_jpg') :
        os.mkdir(path + '_jpg')
        
    # 모든 png 파일의 절대경로를 저장
    all_image_files = glob.glob(path + '/*.png')
    
    # 모든 png파일 경로에 대하여
    for file_path in all_image_files :
        
        # 이미지를 불러온다.
        img = Image.open(file_path).convert("RGB")
        
        # 절대경로상의 모든 디렉토리를 얻어낸다.
        directories = file_path.split('/')
        
        # 저장될 디렉토리의 이름을 지정
        directories[-2] += '_jpg'
        
        # 저장될 파일의 이름 지정
        directories[-1] = directories[-1][:-4] + '.jpg'
        
        # 절대경로명으로 바꾸지
        save_filepath = '/'.join(directories)
        
        # jpg파일을 저장한다.
        img.save(save_filepath, quality=100)
        
folders = ['rock', 'scissor', 'paper']

for fn in folders :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_training/" + fn
    convert_png_to_jpg(image_dir_path)
    
for fn in folders :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_test/" + fn
    convert_png_to_jpg(image_dir_path)

In [37]:
# 이미지 데이터 resize 하기

folders = ['rock_jpg', 'scissor_jpg', 'paper_jpg']

for fn in folders :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_training/" + fn
    print("이미지 디렉토리 경로 : ", image_dir_path)
    
    # 위 디렉토리에 있는 파일 중 .jpg 인 모든 파일 불러오기
    images = glob.glob(image_dir_path + "/*.jpg")
    # print(images)

    # 파일 별 28size 로 resize
    target_size = (28, 28)
    for img in images :
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img,"JPEG")
        # print(new_img)

        # print("모든 이미지 resize 완료.")

이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_training/rock_jpg
이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_training/scissor_jpg
이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_training/paper_jpg


In [38]:
# 훈련용 데이터 셋 만들기

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=2517   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    
    # 이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size,color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path + '/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
        print(file)
        
    for file in glob.iglob(img_path+'/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels
    
folders_1 = ['rock_jpg', 'scissor_jpg', 'paper_jpg']

for fn in folders_1 :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_training/" + fn
    (x_train, y_train)=load_data(image_dir_path)
    x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

/home/aiffel/rock_scissor_paper_training/rock_jpg/rock03-061.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock06ck02-079.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock06ck02-038.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock02-103.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock06ck02-094.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock07-k03-071.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock04-043.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock05ck01-047.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock01-106.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock07-k03-023.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock02-075.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock01-107.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock03-095.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock05ck01-111.jpg
/home/aiffel/rock_scissor_paper_training/rock_jpg/rock01-068.j

학습데이터(x_train)의 이미지 개수는 2517 입니다.
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors01-068.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors02-059.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors03-026.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors02-056.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors03-059.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors04-117.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors03-047.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors03-062.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors01-102.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors01-042.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors01-093.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/testscissors03-088.jpg
/home/aiffel/rock_scissor_paper_training/scissor_jpg/scissors03-078.jpg
/home/

학습데이터(x_train)의 이미지 개수는 2517 입니다.
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper04-111.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper04-119.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper07-049.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper02-075.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper04-023.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper03-032.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper01-085.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper03-023.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper02-009.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper05-035.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper06-112.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper06-030.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper05-076.jpg
/home/aiffel/rock_scissor_paper_training/paper_jpg/paper03-079.jpg
/home/aiffel/rock_scissor_pa

학습데이터(x_train)의 이미지 개수는 2517 입니다.
x_train shape: (2517, 28, 28, 3)
y_train shape: (2517,)


In [39]:
# 딥러닝 네트워크 구성

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (28, 28, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Conv2D(32, (3, 3), activation = "relu"))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

print('Model에 추가된 Layer 개수 : ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수 :  7
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_7 (Dense)              

In [40]:
# 딥러닝 학습하기

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
79/79 [==============================] - 0s 2ms/step - loss: 1.1075 - accuracy: 0.3222
Epoch 2/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0991 - accuracy: 0.3242
Epoch 3/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0995 - accuracy: 0.3321
Epoch 4/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0993 - accuracy: 0.3321
Epoch 5/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0992 - accuracy: 0.3182
Epoch 6/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3159
Epoch 7/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0989 - accuracy: 0.3278
Epoch 8/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3246
Epoch 9/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3325
Epoch 10/10
79/79 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3170


In [41]:
# 테스트 셋으로 확인해보기

In [42]:
# 테스트 데이터 파일 resize 하기

#folders = ['rock', 'scissor', 'paper']

for fn in folders :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_test/" + fn
    print("이미지 디렉토리 경로 : ", image_dir_path)
    
    # 위 디렉토리에 있는 파일 중 .jpg 인 모든 파일 불러오기
    images = glob.glob(image_dir_path + "/*.jpg")
    # print(images)

    # 파일 별 28size 로 resize
    target_size = (28, 28)
    for img in images :
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img,"JPEG")
        # print(new_img)

        # print("모든 이미지 resize 완료.")

이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_test/rock_jpg
이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_test/scissor_jpg
이미지 디렉토리 경로 :  /home/aiffel/rock_scissor_paper_test/paper_jpg


In [44]:
# 테스트 데이터 셋 만들기

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=2517   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    
    # 이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size,color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path + '/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
        print(file)
        
    for file in glob.iglob(img_path+'/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트 데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels
    
folders_1 = ['rock_jpg', 'scissor_jpg', 'paper_jpg']

for fn in folders_1 :
    image_dir_path = os.getenv("HOME") + "/rock_scissor_paper_test/" + fn
    (x_test, y_test)=load_data(image_dir_path)
    x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock04-17.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock03-07.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-13.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock01-15.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-09.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock03-21.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock04-13.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-05.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock03-11.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-00.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock04-05.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-19.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-10.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock01-30.jpg
/home/aiffel/rock_scissor_paper_test/rock_jpg/testrock02-11.jpg
/home/aiffel/rock_scissor_paper_test/roc

In [45]:
# 테스트용 데이터가 준비되었으니, 위에서 훈련시킨 model을 사용하여 test_accuracy를 측정해 봅시다.

In [46]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

79/79 - 0s - loss: 1.0938 - accuracy: 0.9015
test_loss: 1.0938385725021362 
test_accuracy: 0.9014700055122375
